In [ ]:
# Install required packages and download data
%pip install -r requirements.txt
!sh get_data.sh


In [ ]:

!sh clean_data.sh

In [ ]:
import duckdb
import pandas as pd
from os import listdir #, getenv
# from dotenv import load_dotenv
# load_dotenv()

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///cerulean.db

In [ ]:
# Define a function to create a schema for each data source
# and a table for each file (*.csv.gz) in its directory

def create_schema_and_tables(schema, path):
    files = listdir(path)
    %sql CREATE SCHEMA IF NOT EXISTS {schema};
    for file in files:
        if file.endswith('.csv.gz'):
            table = file.split('.')[0]
            %sql DROP TABLE IF EXISTS {schema}.{table};
            %sql CREATE TABLE {schema}.{table} AS SELECT * FROM read_csv('{path}{file}', AUTO_DETECT=TRUE);
            # , NULLSTR='-*');
    %sql result << SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema = '{schema}';
    return result

In [ ]:
schema = 'bad_files'
path = 'bad_files/'

def create_schema_and_tables(schema, path):
    files = listdir(path)
    %sql CREATE SCHEMA IF NOT EXISTS {schema};
    for file in files:
        if file.endswith('.csv.gz'):
            table = file.split('.')[0]
            %sql DROP TABLE IF EXISTS {schema}.{table};
            %sql CREATE TABLE {schema}.{table} AS SELECT * FROM read_csv('{path}{file}', AUTO_DETECT=TRUE);
            # , NULLSTR='-*');
    %sql result << SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema = '{schema}';
    return result

create_schema_and_tables(schema, path)

# # timestamp field value out of range: "" in column "stoptime"
# 'physionet.org/files/mimiciv/2.2/hosp/prescriptions.csv.gz',
# # timestamp field value out of range: "" in column "verifiedtime"
# 'physionet.org/files/mimiciv/2.2/hosp/pharmacy.csv.gz',
# # timestamp field value out of range: "" in column "scheduletime"
# 'physionet.org/files/mimiciv/2.2/hosp/emar.csv.gz',
# # timestamp field value out of range: "" in column "storetime"
# 'physionet.org/files/mimic-iv-note/2.2/note/discharge.csv.gz'

In [ ]:
# Import MIMIC-IV Hospital
schema = 'mimiciv_hosp'
path = 'physionet.org/files/mimiciv/2.2/hosp/'

create_schema_and_tables(schema, path)

In [ ]:
# Import MIMIC-IV ICU
schema = 'mimiciv_icu'
path = 'physionet.org/files/mimiciv/2.2/icu/'

create_schema_and_tables(schema, path)

In [ ]:
# Import MIMIC-IV Notes
schema = 'mimiciv_note'
path = 'physionet.org/files/mimic-iv-note/2.2/note/'

create_schema_and_tables(schema, path)

In [ ]:
# Ad-hoc queries

# %sql df << SELECT * FROM discharge USING SAMPLE 5
# (df['note_id'] + df['text']).to_csv('out/discharge_sample05.csv', index=False)

# %sql df << SELECT * FROM radiology USING SAMPLE 5
# (df['note_id'] + df['text']).to_csv('out/radiology_sample05.csv', index=False)

# df[["text"][0]].to_clipboard()

# %sql df << 
# SELECT d.subject_id, r.subject_id \
#     FROM (SELECT subject_id FROM discharge GROUP BY subject_id) d \
#     LEFT JOIN (SELECT subject_id FROM radiology GROUP BY subject_id) r \
#     ON d.subject_id = r.subject_id